# Import

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
from app.utils.functions import *
from app.core import config
from IPython.display import HTML
from app.services.ollama_service import format_response
import httpx

# Data Preparation

In [2]:
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)

In [24]:
sa_fk = 227

data = await execute_sp(
    "ia.sp_simBudFormSA_one", 
    {
        "user_fk": config.USER_FK, 
        "sa_fk": sa_fk, 
        "form_fk": 167
    }
)
json_string = data[0].get('EcrituresDetails')

data_records = json.loads(json_string)
context_data = pd.DataFrame(data_records)
df = preprocessing_data(context_data, simple_dict)

if df["Section  analytique"].unique().tolist() in [[''], [], None]:
    df["Section  analytique"] = df["Liste de sélection"]
df

,Section analytique,Liste de sélection,Formulaire,Lignes,Type de lignes,Compte reporting,Donnée opérationnelle,Colonnes,Type de colonnes,Montant,Cumul,Nature de l'écriture,Date,Année,Mois,Contexte,Code Hiérarchique,Groupe
0,11400099 - Cergy Saint Christophe,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Réel 2022,Année contexte,61.98,0.0,Mensuelle,2022-01-31,2022.0,1.0,R,4.,Chiffre d'affaire
1,11400099 - Cergy Saint Christophe,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Réel 2022,Année contexte,52.77,0.0,Mensuelle,2022-01-31,2022.0,1.0,R,7.,Chiffre d'affaire
2,11400099 - Cergy Saint Christophe,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Réel 2022,Année contexte,49.24,0.0,Mensuelle,2022-01-31,2022.0,1.0,R,10.,Chiffre d'affaire
3,11400099 - Cergy Saint Christophe,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Réel 2022,Année contexte,72.74,0.0,Mensuelle,2022-02-28,2022.0,2.0,R,4.,Chiffre d'affaire
4,11400099 - Cergy Saint Christophe,,Compte d'exploitation,% DES RECETTES TOTALES,Ligne de calcul,,,Réel 2022,Année contexte,-39.83,0.0,Mensuelle,2022-02-28,2022.0,2.0,R,7.,Chiffre d'affaire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6272,11400099 - Cergy Saint Christophe,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Réel 2024,Année contexte,416.00,0.0,Mensuelle,2024-09-30,2024.0,9.0,R,2.9.3.,Charge
6273,11400099 - Cergy Saint Christophe,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Réel 2024,Année contexte,1391.00,0.0,Annuelle,2024-12-31,2024.0,12.0,R,2.9.3.,Charge
6274,11400099 - Cergy Saint Christophe,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Prév 2025,Année contexte,1035.00,0.0,Mensuelle,2025-01-31,2025.0,1.0,P,2.9.3.,Charge
6275,11400099 - Cergy Saint Christophe,,Compte d'exploitation,Vaisselle logement et cafétaria,Regroupement CR,,,Prév 2025,Année contexte,307.00,0.0,Mensuelle,2025-10-31,2025.0,10.0,P,2.9.3.,Charge


In [26]:
# Renommage et nettoyage
df = df.rename(
    columns={
        'Code Hiérarchique': 'Code_H', 
        'Montant': 'Montant',
        'Lignes': 'Ligne_Analytique',
        'Contexte': 'Contexte',
        'Année': 'Annee',
        'Groupe': 'Groupe',
        'Section  analytique': 'Residence'
    }
)

df['Annee'] = df['Annee'].astype(int)
df['Mois'] = df['Mois'].astype(int)
df['Contexte'] = df['Contexte'].replace({'R': 'Réel', 'B': 'Budget', 'P': 'Prévision'})

df_agg = df.groupby(
    [
        'Residence', 'Colonnes', 'Annee', 'Mois', "Nature de l'écriture", 'Contexte', 'Code_H', 'Ligne_Analytique', 'Groupe'
    ]
)['Montant'].sum().reset_index()

contexte_order = ['Réel', 'Prévision', 'Budget']

def mois_sort_key(mois):
    try:
        return int(mois)
    except:
        return 99

df_pivot = df_agg.pivot_table(
    index=['Groupe', 'Code_H', 'Ligne_Analytique'],
    columns=['Annee', 'Contexte', 'Mois', "Nature de l'écriture"],
    values='Montant',
    fill_value="N/A",
    aggfunc='sum'
)

if df_pivot.columns.nlevels == 4:
    nature_unique = df_pivot.columns.get_level_values(3).unique().tolist()
    if "Annuelle" in nature_unique:
        nature_unique = [n for n in nature_unique if n != "Annuelle"]
        nature_order_desc = sorted(nature_unique, reverse=True) + ["Annuelle"]
    else:
        nature_order_desc = sorted(nature_unique, reverse=True)
    nature_order_dict = {name: i for i, name in enumerate(nature_order_desc)}
    
    def col_sort_key(x):
        return (
            int(x[0]) if str(x[0]).isdigit() else 0,
            contexte_order.index(x[1]) if x[1] in contexte_order else 99,
            mois_sort_key(x[2]),
            nature_order_dict.get(x[3], 999)
        )
    df_pivot = df_pivot[sorted(df_pivot.columns, key=col_sort_key)]

df_pivot = df_pivot.reset_index()

def code_hierarchical_sort_key(code):
    parts = [int(part) if part.isdigit() else part for part in re.split(r'\D+', str(code).strip('.')) if part]
    return parts

df_pivot_sorted = df_pivot.copy()
df_pivot_sorted['__sort_key'] = df_pivot_sorted['Code_H'].apply(code_hierarchical_sort_key)
df_pivot_sorted = df_pivot_sorted.sort_values('__sort_key').drop(columns='__sort_key', level=0).reset_index(drop=True)
df_pivot_sorted

Annee                            Groupe  Code_H  \
Contexte                                          
Mois                                              
Nature de l'écriture                              
0                     Chiffre d'affaire      1.   
1                     Chiffre d'affaire    1.1.   
2                     Chiffre d'affaire  1.1.1.   
3                     Chiffre d'affaire  1.1.4.   
4                     Chiffre d'affaire    1.2.   
..                                  ...     ...   
150                   Chiffre d'affaire     10.   
151                   Chiffre d'affaire     11.   
152                               Marge     12.   
153                              Charge     13.   
154                               Marge     14.   

Annee                                     Ligne_Analytique      2022  \
Contexte                                                        Réel   
Mois                                                               1   
Nature de l'écriture                                       Mensuelle   
0                                                 RECETTES  139485.0   
1                          Loyers logements et parkings HT  126783.0   
2                                     CA Locatif Estudines  126524.0   
3                                      CA Locatif Parkings     259.0   
4                                         RECETTES ANNEXES   12702.0   
..                                                     ...       ...   
150                                 % DES RECETTES TOTALES     49.24   
151                   Dotations aux amortissements (CAPEX)    1400.0   
152                                                 EBITDA   70076.0   
153                                                  CAPEX       N/A   
154                                         FREE CASH FLOW   70076.0   

Annee                                                                   \
Contexte                                                                 
Mois                         2         3         4         5         6   
Nature de l'écriture Mensuelle Mensuelle Mensuelle Mensuelle Mensuelle   
0                     126233.0  135371.0  151317.0  124598.0  116197.0   
1                     123794.0  126596.0  130619.0  124232.0  115369.0   
2                     123480.0  126275.0  130248.0  123910.0  115047.0   
3                        314.0     322.0     371.0     322.0     322.0   
4                       2439.0    8904.0   20699.0     366.0     833.0   
..                         ...       ...       ...       ...       ...   
150                     -40.44    -16.49     73.93    -64.22    -56.42   
151                     1229.0    1563.0    1519.0    1567.0    1599.0   
152                   -49814.0  -20766.0  113381.0  -78451.0  -63961.0   
153                        N/A    6435.0    5634.0    4153.0   17222.0   
154                   -49814.0  -27201.0  107747.0  -82604.0  -81184.0   

Annee                           ...      2025                                \
Contexte                        ...    Budget                                 
Mois                         7  ...         4         5         6         7   
Nature de l'écriture Mensuelle  ... Mensuelle Mensuelle Mensuelle Mensuelle   
0                     130336.0  ...  148935.0  131598.0  127477.0  134691.0   
1                     112515.0  ...  132760.0  129675.0  125542.0  116860.0   
2                     112194.0  ...  132283.0  129199.0  125066.0  116384.0   
3                        322.0  ...     477.0     477.0     477.0     477.0   
4                      17821.0  ...   16176.0    1923.0    1936.0   17831.0   
..                         ...  ...       ...       ...       ...       ...   
150                      45.45  ...     56.64    -54.35    -34.45     51.63   
151                     1835.0  ...    2641.0    2729.0    2641.0    2729.0   
152                    61071.0  ...   86995.0  -68795.0  -41278.0   72266.0   
153            

In [22]:
print(df_pivot_sorted.describe().to_markdown())

|        | ('Groupe', '', '', '')   |   ('Code_H', '', '', '') | ('Ligne_Analytique', '', '', '')   | (2022, 'Réel', 1, 'Mensuelle')   | (2022, 'Réel', 2, 'Mensuelle')   | (2022, 'Réel', 3, 'Mensuelle')   | (2022, 'Réel', 4, 'Mensuelle')   | (2022, 'Réel', 5, 'Mensuelle')   | (2022, 'Réel', 6, 'Mensuelle')   | (2022, 'Réel', 7, 'Mensuelle')   | (2022, 'Réel', 8, 'Mensuelle')   | (2022, 'Réel', 9, 'Mensuelle')   | (2022, 'Réel', 10, 'Mensuelle')   | (2022, 'Réel', 11, 'Mensuelle')   | (2022, 'Réel', 12, 'Mensuelle')   | (2022, 'Réel', 12, 'Annuelle')   | (2023, 'Réel', 1, 'Mensuelle')   | (2023, 'Réel', 2, 'Mensuelle')   | (2023, 'Réel', 3, 'Mensuelle')   | (2023, 'Réel', 4, 'Mensuelle')   | (2023, 'Réel', 5, 'Mensuelle')   | (2023, 'Réel', 6, 'Mensuelle')   | (2023, 'Réel', 7, 'Mensuelle')   | (2023, 'Réel', 8, 'Mensuelle')   | (2023, 'Réel', 9, 'Mensuelle')   | (2023, 'Réel', 10, 'Mensuelle')   | (2023, 'Réel', 11, 'Mensuelle')   | (2023, 'Réel', 12, 'Mensuelle')   | (2023, 'Réel', 12

---

In [18]:
mask_annuelle = df_pivot_sorted.columns.get_level_values(3) == "Annuelle"
annuelle_cols = df_pivot_sorted.columns[mask_annuelle].tolist()

meta_cols = [c for c in df_pivot_sorted.columns if c[0] in ("Groupe", "Code_H", "Ligne_Analytique")]

selected_cols = meta_cols + annuelle_cols
df_pivot_sorted_annual = df_pivot_sorted.loc[:, selected_cols]

def format_value(val):
    try:
        if isinstance(val, str) and val.strip().startswith("%"):
            num = float(val.strip().replace("%", "").replace(",", "."))
            return "{:.2f} %".format(num)

        if isinstance(val, str) and "%" in val:
            num = float(val.replace("%", "").replace(",", ".").strip())
            return "{:.2f} %".format(num)

        if isinstance(val, (float, np.floating, int, np.integer)):
            if float(val) == int(val):
                return int(val)
            else:
                return "{:.2f}".format(float(val))

        if isinstance(val, str):
            num = float(val.replace(",", ".").strip())
            if num == int(num):
                return int(num)
            else:
                return "{:.2f}".format(num)
        return val
    except:
        return val

for col in df_pivot_sorted_annual.columns:
    df_pivot_sorted_annual[col] = df_pivot_sorted_annual[col].apply(format_value)

df_pivot_sorted_annual

Annee                            Groupe  Code_H  \
Contexte                                          
Mois                                              
Nature de l'écriture                              
0                     Chiffre d'affaire       1   
1                     Chiffre d'affaire    1.1.   
2                     Chiffre d'affaire  1.1.1.   
3                     Chiffre d'affaire  1.1.4.   
4                     Chiffre d'affaire    1.2.   
..                                  ...     ...   
150                   Chiffre d'affaire      10   
151                   Chiffre d'affaire      11   
152                               Marge      12   
153                              Charge      13   
154                               Marge      14   

Annee                                     Ligne_Analytique     2022     2023  \
Contexte                                                       Réel     Réel   
Mois                                                             12       12   
Nature de l'écriture                                       Annuelle Annuelle   
0                                                 RECETTES  1606512  1680703   
1                          Loyers logements et parkings HT  1513333  1592405   
2                                     CA Locatif Estudines  1508484  1587237   
3                                      CA Locatif Parkings     4849     5168   
4                                         RECETTES ANNEXES    95883    96244   
..                                                     ...      ...      ...   
150                                 % DES RECETTES TOTALES     0.77     4.27   
151                   Dotations aux amortissements (CAPEX)    19601    23707   
152                                                 EBITDA    31952    95390   
153                                                  CAPEX    50692    42568   
154                                         FREE CASH FLOW   -18741    52822   

Annee                    2024      2025           
Contexte                 Réel Prévision   Budget  
Mois                       12        12       12  
Nature de l'écriture Annuelle  Annuelle Annuelle  
0                     1688848   1455851  1694160  
1                     1574220   1361840  1610479  
2                     1568551   1355007  1604761  
3                        5670      6833     5718  
4                      107871     94663    95344  
..                        ...       ...      ...  
150                     -7.94     -0.33    -4.90  
151                     30475     26205    27761  
152                   -103668     21332   -55291  
153                     32445     49270      N/A  
154                   -136114    -27938   -55291  

[155 rows x 8 columns]

In [5]:
df_pivot_sorted_annual_h1 = df_pivot_sorted_annual[df_pivot_sorted_annual["Code_H"].apply(lambda x: len(str(x).strip('.').split('.')) < 2)].reset_index(drop=True)
if ("Code_H", "", "", "") in df_pivot_sorted_annual_h1.columns:
    df_pivot_sorted_annual_h1 = df_pivot_sorted_annual_h1.drop(columns=[("Code_H", "", "", "")])

df_pivot_sorted_annual_h1

Annee                            Groupe                      Ligne_Analytique  \
Contexte                                                                        
Mois                                                                            
Nature de l'écriture                                                            
0                     Chiffre d'affaire                              RECETTES   
1                                Charge           CHARGES D'IMMEUBLE DIRECTES   
2                                 Marge                               MARGE 1   
3                     Chiffre d'affaire                % DES RECETTES TOTALES   
4                                Charge                               Total 1   
5                                 Marge                               MARGE 2   
6                     Chiffre d'affaire                % DES RECETTES TOTALES   
7                                Charge                               Total 2   
8                                 Marge                               MARGE 3   
9                     Chiffre d'affaire                % DES RECETTES TOTALES   
10                    Chiffre d'affaire  Dotations aux amortissements (CAPEX)   
11                                Marge                                EBITDA   
12                               Charge                                 CAPEX   
13                                Marge                        FREE CASH FLOW   

Annee                    2022     2023     2024      2025           
Contexte                 Réel     Réel     Réel Prévision   Budget  
Mois                       12       12       12        12       12  
Nature de l'écriture Annuelle Annuelle Annuelle  Annuelle Annuelle  
0                     1606512  1680703  1688848   1455851  1694160  
1                      451643   468732   589698    496420   527508  
2                     1154869  1211971  1099150    959431  1166652  
3                       71.89    72.11    65.08     65.90    68.86  
4                     1040538  1014898  1071147    818728  1095321  
5                      114331   197073    28003    140703    71331  
6                        7.12    11.73     1.66      9.66     4.21  
7                      101980   125389   162146    145577   154384  
8                       12351    71684  -134143     -4873   -83052  
9                        0.77     4.27    -7.94     -0.33    -4.90  
10                      19601    23707    30475     26205    27761  
11                      31952    95390  -103668     21332   -55291  
12                      50692    42568    32445     49270      N/A  
13                     -18741    52822  -136114    -27938   -55291

In [6]:
df_pivot_sorted_annual_h2 = df_pivot_sorted_annual[df_pivot_sorted_annual["Code_H"].apply(lambda x: len(str(x).strip('.').split('.')) < 3)].reset_index(drop=True)
if ("Code_H", "", "", "") in df_pivot_sorted_annual_h2.columns:
    df_pivot_sorted_annual_h2 = df_pivot_sorted_annual_h2.drop(columns=[("Code_H", "", "", "")])

df_pivot_sorted_annual_h2

Annee                            Groupe  \
Contexte                                  
Mois                                      
Nature de l'écriture                      
0                     Chiffre d'affaire   
1                     Chiffre d'affaire   
2                     Chiffre d'affaire   
3                     Chiffre d'affaire   
4                     Chiffre d'affaire   
5                                Charge   
6                                Charge   
7                                Charge   
8                                Charge   
9                                Charge   
10                               Charge   
11                               Charge   
12                               Charge   
13                               Charge   
14                               Charge   
15                               Charge   
16                               Charge   
17                               Charge   
18                               Charge   
19                               Charge   
20                               Charge   
21                                Marge   
22                    Chiffre d'affaire   
23                               Charge   
24                               Charge   
25                               Charge   
26                               Charge   
27                               Charge   
28                               Charge   
29                                Marge   
30                    Chiffre d'affaire   
31                               Charge   
32                               Charge   
33                               Charge   
34                               Charge   
35                               Charge   
36                                Marge   
37                    Chiffre d'affaire   
38                    Chiffre d'affaire   
39                                Marge   
40                               Charge   
41                                Marge   

Annee                                             Ligne_Analytique     2022  \
Contexte                                                               Réel   
Mois                                                                     12   
Nature de l'écriture                                               Annuelle   
0                                                         RECETTES  1606512   
1                                  Loyers logements et parkings HT  1513333   
2                                                 RECETTES ANNEXES    95883   
3                                             Commission Agence TO     -936   
4                                                          IMPAYES    -1769   
5                                      CHARGES D'IMMEUBLE DIRECTES   451643   
6                                               FRAIS DE PERSONNEL   150920   
7                                         IMPOTS et TAXES DIVERSES     3879   
8                                                          ENERGIE   156229   
9                                          CONTRATS DE MAINTENANCE    41173   
10                               REMISE EN ETAT PARTIES PRIVATIVES    27648   
11                                 REMISE EN ETAT PARTIES COMMUNES    23990   
12                                           LINGE ET BLANCHISSAGE     5477   
13                                      FOURNITURES D'EXPLOITATION    11277   
14                                           ACHAT ALIMENTAIRE PDJ        0   
15                              LOCATIONS, HONORAIRES, SACEM ET TV     6288   
16                                          DEPLACEMENT ET MISSION      550   
17                                                       PUBLICITE     3834   
18                                              Publicité Internet     2387   
19                                     FOURNITURES ADMINISTRATIVES      930   
20                            TELEPHONIE - INTERNET - INFORMATIQUE    17061   
21                                            

---

In [7]:
current_year = datetime.now().year
years = [current_year-2, current_year-1, current_year]

meta_cols = df_pivot_sorted_annual_h1.columns[:2]
year_cols = [col for col in df_pivot_sorted_annual_h1.columns[2:] if col[0] in years]
cols_to_keep = list(meta_cols) + year_cols

df_pivot_sorted_annual_h1_the_years = df_pivot_sorted_annual_h1.loc[:, cols_to_keep]
df_pivot_sorted_annual_h1_the_years

Annee                            Groupe                      Ligne_Analytique  \
Contexte                                                                        
Mois                                                                            
Nature de l'écriture                                                            
0                     Chiffre d'affaire                              RECETTES   
1                                Charge           CHARGES D'IMMEUBLE DIRECTES   
2                                 Marge                               MARGE 1   
3                     Chiffre d'affaire                % DES RECETTES TOTALES   
4                                Charge                               Total 1   
5                                 Marge                               MARGE 2   
6                     Chiffre d'affaire                % DES RECETTES TOTALES   
7                                Charge                               Total 2   
8                                 Marge                               MARGE 3   
9                     Chiffre d'affaire                % DES RECETTES TOTALES   
10                    Chiffre d'affaire  Dotations aux amortissements (CAPEX)   
11                                Marge                                EBITDA   
12                               Charge                                 CAPEX   
13                                Marge                        FREE CASH FLOW   

Annee                    2023     2024      2025           
Contexte                 Réel     Réel Prévision   Budget  
Mois                       12       12        12       12  
Nature de l'écriture Annuelle Annuelle  Annuelle Annuelle  
0                     1680703  1688848   1455851  1694160  
1                      468732   589698    496420   527508  
2                     1211971  1099150    959431  1166652  
3                       72.11    65.08     65.90    68.86  
4                     1014898  1071147    818728  1095321  
5                      197073    28003    140703    71331  
6                       11.73     1.66      9.66     4.21  
7                      125389   162146    145577   154384  
8                       71684  -134143     -4873   -83052  
9                        4.27    -7.94     -0.33    -4.90  
10                      23707    30475     26205    27761  
11                      95390  -103668     21332   -55291  
12                      42568    32445     49270      N/A  
13                      52822  -136114    -27938   -55291

In [8]:
current_year = datetime.now().year
years = [current_year-2, current_year-1, current_year]

df_pivot_sorted_annual_h2_Charge = df_pivot_sorted_annual[
    df_pivot_sorted_annual["Code_H"].apply(
        lambda x: (len(str(x).strip('.').split('.')) <= 2) and (str(x).strip('.').split('.')[0] in ['2', '5', '8', '13'])
    )
].reset_index(drop=True)

meta_cols = df_pivot_sorted_annual_h2_Charge.columns[:3]
year_cols = [col for col in df_pivot_sorted_annual_h1.columns[2:] if col[0] in years]
cols_to_keep = list(meta_cols) + year_cols

df_pivot_sorted_annual_h2_the_years_Charge = df_pivot_sorted_annual_h2_Charge.loc[:, cols_to_keep]
df_pivot_sorted_annual_h2_the_years_Charge

Annee                 Groupe Code_H  \
Contexte                              
Mois                                  
Nature de l'écriture                  
0                     Charge      2   
1                     Charge   2.1.   
2                     Charge   2.2.   
3                     Charge   2.3.   
4                     Charge   2.4.   
5                     Charge   2.6.   
6                     Charge   2.7.   
7                     Charge   2.8.   
8                     Charge   2.9.   
9                     Charge  2.10.   
10                    Charge  2.12.   
11                    Charge  2.13.   
12                    Charge  2.14.   
13                    Charge  2.15.   
14                    Charge  2.16.   
15                    Charge  2.17.   
16                    Charge      5   
17                    Charge   5.1.   
18                    Charge   5.2.   
19                    Charge   5.3.   
20                    Charge   5.4.   
21                    Charge   5.5.   
22                    Charge      8   
23                    Charge   8.1.   
24                    Charge   8.2.   
25                    Charge   8.3.   
26                    Charge   8.4.   
27                    Charge     13   

Annee                                             Ligne_Analytique     2023  \
Contexte                                                               Réel   
Mois                                                                     12   
Nature de l'écriture                                               Annuelle   
0                                      CHARGES D'IMMEUBLE DIRECTES   468732   
1                                               FRAIS DE PERSONNEL   149172   
2                                         IMPOTS et TAXES DIVERSES     1955   
3                                                          ENERGIE   195159   
4                                          CONTRATS DE MAINTENANCE    46202   
5                                REMISE EN ETAT PARTIES PRIVATIVES    10864   
6                                  REMISE EN ETAT PARTIES COMMUNES     7962   
7                                            LINGE ET BLANCHISSAGE     7453   
8                                       FOURNITURES D'EXPLOITATION     6534   
9                                            ACHAT ALIMENTAIRE PDJ       91   
10                              LOCATIONS, HONORAIRES, SACEM ET TV    19949   
11                                          DEPLACEMENT ET MISSION      289   
12                                                       PUBLICITE     2685   
13                                              Publicité Internet     2185   
14                                     FOURNITURES ADMINISTRATIVES      970   
15                            TELEPHONIE - INTERNET - INFORMATIQUE    17261   
16                                                         Total 1  1014898   
17                                    AMORTISSEMENT et CREDIT BAIL    27294   
18                                   LOYERS REVERSES PROPRIETAIRES   851433   
19                                                    GROS TRAVAUX     9735   
20                                                 INCIDENCE T.V.A   113613   
21                                               CET (ex Taxe Pro)    12824   
22                                                         Total 2   125389   
23                    PRESTATIONS INTERNES (Y/C Honoraires syndic)   101978   
24                                              FRAIS DES SOCIETES     7184   
25                                               HONORAIRES SYNDIC    17477   
26                                         CHARGES EXCEPTIONNELLES    -1250   
27                                                           CAPEX    42568   

Annee                    2024      2025           
Contexte                 Réel Prévision   Budget  
Mois                       12        12       12  
Nature de l'écriture Annuelle  Annuelle Annuelle  
0                 

---

In [9]:
current_year = datetime.now().year
years = [current_year-1, current_year]

df_pivot_sorted_annual_h3_CA = df_pivot_sorted_annual[
    df_pivot_sorted_annual["Code_H"].apply(
        lambda x: (len(str(x).strip('.').split('.')) <= 3) and (str(x).strip('.').split('.')[0] == '1')
    )
].reset_index(drop=True)

meta_cols = df_pivot_sorted_annual_h3_CA.columns[:3]
year_cols = [col for col in df_pivot_sorted_annual_h1.columns[2:] if col[0] in years]
cols_to_keep = list(meta_cols) + year_cols

df_pivot_sorted_annual_h3_the_years_CA = df_pivot_sorted_annual_h3_CA.loc[:, cols_to_keep]
df_pivot_sorted_annual_h3_the_years_CA

Annee                            Groupe   Code_H  \
Contexte                                           
Mois                                               
Nature de l'écriture                               
0                     Chiffre d'affaire        1   
1                     Chiffre d'affaire     1.1.   
2                     Chiffre d'affaire   1.1.1.   
3                     Chiffre d'affaire   1.1.4.   
4                     Chiffre d'affaire     1.2.   
5                     Chiffre d'affaire   1.2.2.   
6                     Chiffre d'affaire   1.2.4.   
7                     Chiffre d'affaire   1.2.5.   
8                     Chiffre d'affaire   1.2.8.   
9                     Chiffre d'affaire   1.2.9.   
10                    Chiffre d'affaire  1.2.12.   
11                    Chiffre d'affaire     1.3.   
12                    Chiffre d'affaire     1.4.   
13                    Chiffre d'affaire   1.4.1.   
14                    Chiffre d'affaire   1.4.2.   

Annee                                         Ligne_Analytique     2024  \
Contexte                                                           Réel   
Mois                                                                 12   
Nature de l'écriture                                           Annuelle   
0                                                     RECETTES  1688848   
1                              Loyers logements et parkings HT  1574220   
2                                         CA Locatif Estudines  1568551   
3                                          CA Locatif Parkings     5670   
4                                             RECETTES ANNEXES   107871   
5                                             CA Frais locatif    21935   
6                                              CA Restauration      N/A   
7                                            CA Laverie-Ménage    15961   
8                                                    CA Divers      987   
9                     Subvention d'exploitation (Rbst format°)      164   
10                               Charges de copro récupérables    68823   
11                                        Commission Agence TO     -362   
12                                                     IMPAYES     7119   
13                                          Impayés locataires    10857   
14                                        Impayés Apparthotels    -3738   

Annee                     2025           
Contexte             Prévision   Budget  
Mois                        12       12  
Nature de l'écriture  Annuelle Annuelle  
0                      1455851  1694160  
1                      1361840  1610479  
2                      1355007  1604761  
3                         6833     5718  
4                        94663    95344  
5                        12528    19195  
6                          N/A      N/A  
7                        11894    13632  
8                          798     2230  
9                          N/A      N/A  
10                       69442    60288  
11                        -652     -746  
12                         N/A   -10917  
13                         N/A   -10917  
14                         N/A      N/A

In [10]:
current_year = datetime.now().year
years = [current_year-2, current_year-1, current_year]

df_pivot_sorted_annual_h2_RR = df_pivot_sorted_annual[
    df_pivot_sorted_annual["Code_H"].apply(
        lambda x: (len(str(x).strip('.').split('.')) <= 2) and (str(x).strip('.').split('.')[0] in ['1', '2', '3', '4'])
    )
].reset_index(drop=True)

meta_cols = df_pivot_sorted_annual_h2_RR.columns[:3]
year_cols = [col for col in df_pivot_sorted_annual_h1.columns[2:] if col[0] in years]
cols_to_keep = list(meta_cols) + year_cols

df_pivot_sorted_annual_h2_RR_the_years = df_pivot_sorted_annual_h2_RR.loc[:, cols_to_keep]
df_pivot_sorted_annual_h2_RR_the_years

Annee                            Groupe Code_H  \
Contexte                                         
Mois                                             
Nature de l'écriture                             
0                     Chiffre d'affaire      1   
1                     Chiffre d'affaire   1.1.   
2                     Chiffre d'affaire   1.2.   
3                     Chiffre d'affaire   1.3.   
4                     Chiffre d'affaire   1.4.   
5                                Charge      2   
6                                Charge   2.1.   
7                                Charge   2.2.   
8                                Charge   2.3.   
9                                Charge   2.4.   
10                               Charge   2.6.   
11                               Charge   2.7.   
12                               Charge   2.8.   
13                               Charge   2.9.   
14                               Charge  2.10.   
15                               Charge  2.12.   
16                               Charge  2.13.   
17                               Charge  2.14.   
18                               Charge  2.15.   
19                               Charge  2.16.   
20                               Charge  2.17.   
21                                Marge      3   
22                    Chiffre d'affaire      4   

Annee                                     Ligne_Analytique     2023     2024  \
Contexte                                                       Réel     Réel   
Mois                                                             12       12   
Nature de l'écriture                                       Annuelle Annuelle   
0                                                 RECETTES  1680703  1688848   
1                          Loyers logements et parkings HT  1592405  1574220   
2                                         RECETTES ANNEXES    96244   107871   
3                                     Commission Agence TO    -1776     -362   
4                                                  IMPAYES    -6170     7119   
5                              CHARGES D'IMMEUBLE DIRECTES   468732   589698   
6                                       FRAIS DE PERSONNEL   149172   204514   
7                                 IMPOTS et TAXES DIVERSES     1955     1553   
8                                                  ENERGIE   195159   219778   
9                                  CONTRATS DE MAINTENANCE    46202    46885   
10                       REMISE EN ETAT PARTIES PRIVATIVES    10864    29927   
11                         REMISE EN ETAT PARTIES COMMUNES     7962    34669   
12                                   LINGE ET BLANCHISSAGE     7453     8933   
13                              FOURNITURES D'EXPLOITATION     6534     8727   
14                                   ACHAT ALIMENTAIRE PDJ       91        0   
15                      LOCATIONS, HONORAIRES, SACEM ET TV    19949     8253   
16                                  DEPLACEMENT ET MISSION      289      303   
17                                               PUBLICITE     2685     2549   
18                                      Publicité Internet     2185     2345   
19                             FOURNITURES ADMINISTRATIVES      970      887   
20                    TELEPHONIE - INTERNET - INFORMATIQUE    17261    20373   
21                                                 MARGE 1  1211971  1099150   
22                                  % DES RECETTES TOTALES    72.11    65.08   

Annee                     2025           
Contexte             Prévision   Budget  
Mois                        12       12  
Nature de l'écriture  Annuelle Annuelle  
0                      1455851  1694160  
1                      1361840  1610479  
2                        94663    95344  
3                         -652     -746  
4                          N/A   -10917  
5                       496420   527508  
6                       197219   208541  
7                         

In [11]:
current_year = datetime.now().year
years = [current_year-1, current_year]

df_pivot_sorted_annual_h2_Charge_RR = df_pivot_sorted_annual[
    df_pivot_sorted_annual["Code_H"].apply(
        lambda x: (len(str(x).strip('.').split('.')) <= 2) and (str(x).strip('.').split('.')[0] in ['2'])
    )
].reset_index(drop=True)

meta_cols = df_pivot_sorted_annual_h2_Charge_RR.columns[:3]
year_cols = [col for col in df_pivot_sorted_annual_h1.columns[2:] if col[0] in years]
cols_to_keep = list(meta_cols) + year_cols

df_pivot_sorted_annual_h2_the_years_Charge_RR = df_pivot_sorted_annual_h2_Charge_RR.loc[:, cols_to_keep]
df_pivot_sorted_annual_h2_the_years_Charge_RR

Annee                 Groupe Code_H                      Ligne_Analytique  \
Contexte                                                                    
Mois                                                                        
Nature de l'écriture                                                        
0                     Charge      2           CHARGES D'IMMEUBLE DIRECTES   
1                     Charge   2.1.                    FRAIS DE PERSONNEL   
2                     Charge   2.2.              IMPOTS et TAXES DIVERSES   
3                     Charge   2.3.                               ENERGIE   
4                     Charge   2.4.               CONTRATS DE MAINTENANCE   
5                     Charge   2.6.     REMISE EN ETAT PARTIES PRIVATIVES   
6                     Charge   2.7.       REMISE EN ETAT PARTIES COMMUNES   
7                     Charge   2.8.                 LINGE ET BLANCHISSAGE   
8                     Charge   2.9.            FOURNITURES D'EXPLOITATION   
9                     Charge  2.10.                 ACHAT ALIMENTAIRE PDJ   
10                    Charge  2.12.    LOCATIONS, HONORAIRES, SACEM ET TV   
11                    Charge  2.13.                DEPLACEMENT ET MISSION   
12                    Charge  2.14.                             PUBLICITE   
13                    Charge  2.15.                    Publicité Internet   
14                    Charge  2.16.           FOURNITURES ADMINISTRATIVES   
15                    Charge  2.17.  TELEPHONIE - INTERNET - INFORMATIQUE   

Annee                    2024      2025           
Contexte                 Réel Prévision   Budget  
Mois                       12        12       12  
Nature de l'écriture Annuelle  Annuelle Annuelle  
0                      589698    496420   527508  
1                      204514    197219   208541  
2                        1553       238     3247  
3                      219778    165589   163587  
4                       46885     38767    57144  
5                       29927     18252    27600  
6                       34669     36615      N/A  
7                        8933      5043    10313  
8                        8727      7880     6022  
9                           0         0      N/A  
10                       8253      9701    16628  
11                        303       112     2273  
12                       2549      1894     4103  
13                       2345      2263     4452  
14                        887       338      707  
15                      20373     12508    22892

# Ask Ollama

### Ollama Class

In [12]:
class OllamaClient:
    def __init__(self, ollama_url: str):
        self.client = httpx.AsyncClient(timeout=1000)
        self.url = ollama_url

    async def ask_ollama(self, prompt: str):
        payload = {
            "model": config.GPT,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "stream": False,
            "keep_alive": -1,
            "options": {
                "temperature": 0.1,
            }
        }

        response = await self.client.post(
            url=self.url, 
            json=payload
        )
        
        response.raise_for_status() 
        json_data = response.json() 
        if "message" in json_data and "content" in json_data["message"]: 
            content: str = json_data["message"]["content"] 
            parts = content.split("</think>", 1) 
            if len(parts) > 1: 
                content = parts[1] 
            else: 
                content = content 
            html_text = format_response(content)
            
            return html_text, content

ollamaClient = OllamaClient(config.OLLAMA_URL)

### To Markdown

In [23]:
def _format_euro_fr(x: float, line: str) -> str:
    """Format number in French style with 2 decimals and a non-breaking space thousands separator."""
    if pd.isna(x):
            return "N/A"
    elif str(line).startswith("%"):
        s = f"{x:,.2f}"
        s = s.replace(",", " ")
        return f"{s} %"
    else:
        s = f"{x:,.0f}"
        s = s.replace(",", " ")
        return f"{s} €"

def _normalize_col(col):
    if isinstance(col, tuple):
        if len(col) >= 1:
            year = str(col[0])
        else:
            year = "unknown"
        # find Réel/Prévision/Budget if present in tuple (also accent-insensitive, lowercase!)
        typ = next(
            (str(x) for x in col if isinstance(x, str) and str(x).lower() in ("réel", "budget", "prévision", "prevision")),
            None
        )
        if typ is None:
            # search for any string in tuple as fallback
            typ = next((str(x) for x in col if isinstance(x, str)), "Réel")
        return {"year": year, "type": typ}
    else:
        # If col is not a tuple, fallback as string
        year = str(col)
        return {"year": year, "type": "Réel"}

def transform_for_llm(df_pivot: pd.DataFrame):
    """
    Returns:
        - df_long: DataFrame with columns ['Indicateur','Année','Type','Valeur']
        - metric_text: str with metric blocks optimized for LLM input (with custom column order)
        - metric_json: dict structured as {INDICATOR: {period_key: numeric_value}}
    """
    df = df_pivot.copy()

    # Ensure index is a normal column named 'Indicateur' if index holds labels
    if df.index.name is None or df.index.name == "":
        df = df.reset_index()
    # Try to detect indicator column name
    indicator_col = None
    for possible in ['Ligne_Analytique', 'Indicateur', 'index', 0]:
        if possible in df.columns:
            indicator_col = possible
            break
    if indicator_col is None:
        indicator_col = df.columns[0]
    df = df.rename(columns={indicator_col: 'Indicateur'})

    value_cols = [c for c in df.columns if c != 'Indicateur']

    rows = []
    for _, row in df.iterrows():
        indicator = row['Indicateur']
        for col in value_cols:
            meta = _normalize_col(col)
            year = meta['year']
            typ = meta['type']
            try:
                val = row[col]
            except Exception:
                val = row.get(col, None)
            # Try to keep numeric
            numeric = None
            if pd.api.types.is_numeric_dtype(type(val)):
                try:
                    numeric = float(val) if not pd.isna(val) else None
                except Exception:
                    numeric = None
            else:
                try:
                    numeric = float(str(val).replace("€", "").replace("%", "").replace(" ", "").replace(",", "."))
                except Exception:
                    numeric = None
            label = "  ".join([str(x).strip() for x in indicator if x is not None])
            rows.append({
                'Indicateur': indicator,
                'Année': year,
                'Type': typ,
                'Valeur_num': numeric,
                'Valeur_txt': _format_euro_fr(numeric, label) if numeric is not None else "N/A"
            })

    df_long = pd.DataFrame(rows)
    #--- Custom LLM string format ---

    # Define context order mapping (for sorting)
    def context_rank(typ):
        t = str(typ).lower()
        if "réel" in t:
            return 0
        if "prevision" in t or "prévision" in t:
            return 1
        if "budget" in t:
            return 2
        return 99

    # Custom block for indicator
    def _block_for_indicator(ind):
        if isinstance(ind, (list, tuple, pd.Series, np.ndarray)):
            # Remove "Name" and dtype printing, get just the values
            label = "  ".join([str(x).strip() for x in ind if x is not None])
        else:
            label = str(ind).strip()
        lines = [f"[{label}]"]
        sub = df_long[df_long['Indicateur'].astype(str).values == str(ind)].copy()
        # Only keep valid numerical values; skip technical columns
        sub = sub[~((sub['Année'].str.lower() == 'groupe') | (sub['Année'].str.lower() == 'indicateur') | (sub['Année'].str.lower() == 'index'))]
        # Custom sort: context (réel - prévision - budget), then by year
        sub_sorted = sub.copy()
        try:
            sub_sorted["Année_num"] = pd.to_numeric(sub_sorted["Année"], errors='coerce')
        except Exception:
            sub_sorted["Année_num"] = sub_sorted["Année"]
        sub_sorted = sub_sorted.sort_values(by=["Année_num", "Type"], key=lambda col: col, ascending=[True, True])
        sub_sorted = sub_sorted.sort_values(
            by=["Type", "Année_num"],
            key=lambda col: col.map(context_rank) if col.name == "Type" else col,
            ascending=[True, True]
        )
        # Ensure correct order: Réel, Prévision, Budget
        # Gather existing contexts by year
        entries = []
        for ctx in ["Réel", "Prévision", "Budget"]:
            sub_ctx = sub_sorted[sub_sorted["Type"].str.lower().str.contains(ctx.lower())]
            entries.append(sub_ctx)
        merged = pd.concat(entries)
        # Remove duplicates in case some contexts are missing
        merged = merged.drop_duplicates(subset=["Année", "Type"])

        # Yield custom lines
        for _, rr in merged.iterrows():
            year = rr['Année']
            typ = rr['Type']
            txt = rr['Valeur_txt']
            # skip empty technicals, or keep only if year is really a year
            if (year is None or str(year).lower() in ['groupe', 'indicateur', 'index']) and (txt == "0" or txt == "0 €" or txt == "N/A"):
                continue
            # Always print 0, even for Groupe/Indicateur/Index, if present and value not 0?
            lines.append(f"- {year} {typ} : {txt}")
        return "\n".join(lines)

    # Only indicateur list (avoid 'Groupe Groupe', etc.)
    indicators = df_long['Indicateur'].drop_duplicates().tolist()
    blocks = [_block_for_indicator(ind) for ind in indicators]

    metric_text = "\n\n".join(blocks)

    return metric_text

def transform_for_llm_(df_pivot: pd.DataFrame):
    """
    Returns:
        - metric_text: str with metric blocks optimized for LLM input
    Notes:
        - This version detects if contextual columns (e.g. 'Groupe', 'Code_H', 'Ligne_Analytique')
            exist and builds a consolidated 'Indicateur' label from them.
        - Assumes helper functions _normalize_col and _format_euro_fr already exist.
    """
    df = df_pivot.copy()

    # If index contains labels, reset to columns
    if df.index.name is None or df.index.name == "":
        df = df.reset_index()

    # If there are contextual columns, build a consolidated 'Indicateur' column
    context_cols = [c for c in ['Code_H', 'Ligne_Analytique', 'Indicateur'] if c in df.columns]

    if len(context_cols) > 1:
        # create a single descriptive indicator by joining available context columns (in order)
        df['Indicateur_consolide'] = df[context_cols].astype(str).apply(
            lambda row: " | ".join([str(x).strip() for x in row.values if str(x).strip() not in ['nan', 'None']]),
            axis=1
        )
        # prefer the consolidated name
        indicator_col = 'Indicateur_consolide'
    else:
        # detect a single indicator column if present, otherwise use first column
        indicator_col = None
        for possible in ['Ligne_Analytique', 'Indicateur', 'index', 0]:
            if possible in df.columns:
                indicator_col = possible
                break
        if indicator_col is None:
            indicator_col = df.columns[0]
        # if chosen indicator_col isn't already a string label, coerce to str
        if indicator_col != 'Indicateur':
            df[indicator_col] = df[indicator_col].astype(str)

    # Ensure the DataFrame has a column named exactly 'Indicateur' used downstream
    if indicator_col != 'Indicateur':
        df = df.rename(columns={indicator_col: 'Indicateur'})
    else:
        # if it already is 'Indicateur', ensure string type
        df['Indicateur'] = df['Indicateur'].astype(str)

    value_cols = [c for c in df.columns if c != 'Indicateur']

    rows = []
    for _, row in df.iterrows():
        # Avoid pandas row pretty-print for the indicator label
        if isinstance(row['Indicateur'], pd.Series):
            indicator_label = " | ".join(str(x).strip() for x in row['Indicateur'].values if str(x).strip() not in ['nan', 'None'])
        else:
            indicator_label = str(row['Indicateur']).strip()
        for col in value_cols:
            meta = _normalize_col(col)
            year = meta.get('year', 'unknown')
            typ = meta.get('type', 'Réel')
            try:
                val = row[col]
            except Exception:
                val = row.get(col, None)

            # Try to keep numeric
            numeric = None
            if pd.api.types.is_numeric_dtype(type(val)):
                try:
                    numeric = float(val) if not pd.isna(val) else None
                except Exception:
                    numeric = None
            else:
                try:
                    numeric = float(str(val).replace("€", "").replace("%", "").replace(" ", "").replace(",", "."))
                except Exception:
                    numeric = None

            lbl = indicator_label.split(" | ")[-1]
            txt = _format_euro_fr(numeric, lbl) if numeric is not None else "N/A"

            rows.append({
                'Indicateur': indicator_label,
                'Année': year,
                'Type': typ,
                'Valeur_num': numeric,
                'Valeur_txt': txt
            })

    df_long = pd.DataFrame(rows)

    # --- Custom LLM string format ---

    # Define context order mapping (for sorting)
    def context_rank(typ):
        t = str(typ).lower()
        if "réel" in t:
            return 0
        if "prevision" in t or "prévision" in t:
            return 1
        if "budget" in t:
            return 2
        return 99

    def _block_for_indicator(ind):
        label = str(ind).strip()
        lines = [f"[{label}]"]
        sub = df_long[df_long['Indicateur'].astype(str).values == str(ind)].copy()

        # Filter out technical garbage in 'Année'
        sub = sub[~sub['Année'].astype(str).str.lower().isin(['groupe', 'indicateur', 'index'])]

        # Attempt numeric year conversion for sorting; fallback keeps original order
        sub_sorted = sub.copy()
        try:
            sub_sorted["Année_num"] = pd.to_numeric(sub_sorted["Année"], errors='coerce')
        except Exception:
            sub_sorted["Année_num"] = sub_sorted["Année"]

        # First sort by Type context order, then by year
        if 'Type' in sub_sorted.columns:
            sub_sorted = sub_sorted.sort_values(
                by=["Type", "Année_num"],
                key=lambda col: col.map(context_rank) if col.name == "Type" else col,
                ascending=[True, True]
            )
        else:
            sub_sorted = sub_sorted.sort_values(by=["Année_num"])

        # Ensure ordering Réel -> Prévision -> Budget within each year
        entries = []
        for ctx in ["Réel", "Prévision", "Budget"]:
            sub_ctx = sub_sorted[sub_sorted["Type"].astype(str).str.lower().str.contains(ctx.lower(), na=False)]
            entries.append(sub_ctx)
        if entries:
            merged = pd.concat(entries)
            merged = merged.drop_duplicates(subset=["Année", "Type"])
        else:
            merged = sub_sorted

        # Produce lines
        for _, rr in merged.iterrows():
            year = rr['Année']
            typ = rr['Type']
            txt = rr['Valeur_txt']
            # Skip rows with completely empty or N/A values for non-year labels
            if (pd.isna(year) or str(year).strip().lower() in ['nan', 'none', '']) and txt in ["0", "0 €", "N/A"]:
                continue
            lines.append(f"- {year} {typ} : {txt}")
        return "\n".join(lines)

    indicators = df_long['Indicateur'].drop_duplicates().tolist()
    blocks = [_block_for_indicator(ind) for ind in indicators]

    metric_text = "\n\n".join(blocks)

    return metric_text


##### Global

In [14]:
metric_text_g = transform_for_llm_(df_pivot_sorted_annual_h2_RR_the_years)
print(count_tokens(metric_text_g))
print(metric_text_g)

866
[1 | RECETTES]
- 2023 Réel : 1 680 703 €
- 2024 Réel : 1 688 848 €
- 2025 Prévision : 1 455 851 €
- 2025 Budget : 1 694 160 €

[1.1. | Loyers logements et parkings HT]
- 2023 Réel : 1 592 405 €
- 2024 Réel : 1 574 220 €
- 2025 Prévision : 1 361 840 €
- 2025 Budget : 1 610 479 €

[1.2. | RECETTES ANNEXES]
- 2023 Réel : 96 244 €
- 2024 Réel : 107 871 €
- 2025 Prévision : 94 663 €
- 2025 Budget : 95 344 €

[1.3. | Commission Agence TO]
- 2023 Réel : -1 776 €
- 2024 Réel : -362 €
- 2025 Prévision : -652 €
- 2025 Budget : -746 €

[1.4. | IMPAYES]
- 2023 Réel : -6 170 €
- 2024 Réel : 7 119 €
- 2025 Prévision : N/A
- 2025 Budget : -10 917 €

[2 | CHARGES D'IMMEUBLE DIRECTES]
- 2023 Réel : 468 732 €
- 2024 Réel : 589 698 €
- 2025 Prévision : 496 420 €
- 2025 Budget : 527 508 €

[2.1. | FRAIS DE PERSONNEL]
- 2023 Réel : 149 172 €
- 2024 Réel : 204 514 €
- 2025 Prévision : 197 219 €
- 2025 Budget : 208 541 €

[2.2. | IMPOTS et TAXES DIVERSES]
- 2023 Réel : 1 955 €
- 2024 Réel : 1 553 €
- 202

##### Recette

In [15]:
metric_text_ca = transform_for_llm_(df_pivot_sorted_annual_h3_the_years_CA)
print(count_tokens(metric_text_ca))
print(metric_text_ca)

492
[1 | RECETTES]
- 2024 Réel : 1 688 848 €
- 2025 Prévision : 1 455 851 €
- 2025 Budget : 1 694 160 €

[1.1. | Loyers logements et parkings HT]
- 2024 Réel : 1 574 220 €
- 2025 Prévision : 1 361 840 €
- 2025 Budget : 1 610 479 €

[1.1.1. | CA Locatif Estudines]
- 2024 Réel : 1 568 551 €
- 2025 Prévision : 1 355 007 €
- 2025 Budget : 1 604 761 €

[1.1.4. | CA Locatif Parkings]
- 2024 Réel : 5 670 €
- 2025 Prévision : 6 833 €
- 2025 Budget : 5 718 €

[1.2. | RECETTES ANNEXES]
- 2024 Réel : 107 871 €
- 2025 Prévision : 94 663 €
- 2025 Budget : 95 344 €

[1.2.2. | CA Frais locatif]
- 2024 Réel : 21 935 €
- 2025 Prévision : 12 528 €
- 2025 Budget : 19 195 €

[1.2.4. | CA Restauration]
- 2024 Réel : N/A
- 2025 Prévision : N/A
- 2025 Budget : N/A

[1.2.5. | CA Laverie-Ménage]
- 2024 Réel : 15 961 €
- 2025 Prévision : 11 894 €
- 2025 Budget : 13 632 €

[1.2.8. | CA Divers]
- 2024 Réel : 987 €
- 2025 Prévision : 798 €
- 2025 Budget : 2 230 €

[1.2.9. | Subvention d'exploitation (Rbst format°)

##### Charge

In [16]:
metric_text_charge = transform_for_llm_(df_pivot_sorted_annual_h2_the_years_Charge_RR)
print(count_tokens(metric_text_charge))
print(metric_text_charge)

493
[2 | CHARGES D'IMMEUBLE DIRECTES]
- 2024 Réel : 589 698 €
- 2025 Prévision : 496 420 €
- 2025 Budget : 527 508 €

[2.1. | FRAIS DE PERSONNEL]
- 2024 Réel : 204 514 €
- 2025 Prévision : 197 219 €
- 2025 Budget : 208 541 €

[2.2. | IMPOTS et TAXES DIVERSES]
- 2024 Réel : 1 553 €
- 2025 Prévision : 238 €
- 2025 Budget : 3 247 €

[2.3. | ENERGIE]
- 2024 Réel : 219 778 €
- 2025 Prévision : 165 589 €
- 2025 Budget : 163 587 €

[2.4. | CONTRATS DE MAINTENANCE]
- 2024 Réel : 46 885 €
- 2025 Prévision : 38 767 €
- 2025 Budget : 57 144 €

[2.6. | REMISE EN ETAT PARTIES PRIVATIVES]
- 2024 Réel : 29 927 €
- 2025 Prévision : 18 252 €
- 2025 Budget : 27 600 €

[2.7. | REMISE EN ETAT PARTIES COMMUNES]
- 2024 Réel : 34 669 €
- 2025 Prévision : 36 615 €
- 2025 Budget : N/A

[2.8. | LINGE ET BLANCHISSAGE]
- 2024 Réel : 8 933 €
- 2025 Prévision : 5 043 €
- 2025 Budget : 10 313 €

[2.9. | FOURNITURES D'EXPLOITATION]
- 2024 Réel : 8 727 €
- 2025 Prévision : 7 880 €
- 2025 Budget : 6 022 €

[2.10. | ACH

### Prompt

##### Global

In [18]:
current_year = datetime.now().year
PROMPT_G = f"""
Tu es analyste financier senior spécialisé en exploitation de résidences étudiantes.

DONNÉES FOURNIES - TABLEAUX DE BORD OPÉRATIONNELS
{metric_text_g}

RÈGLES STRICTES
- Analyse uniquement à partir des données fournies.
- Aucune hypothèse, extrapolation ou interprétation externe.
- Valeur prioritaire : Réel > Prévision > Budget.

RÈGLE RÉEL / PRÉVISION
- Janvier à août : Prévision = Réel.
- Septembre à décembre : Prévision = projection du Réel.

AGRÉGATS ET FORMULES (DÉJÀ INTÉGRÉS)
- Marge 1 = Recettes - Charges d'immeuble directes.
- % DES RECETTES TOTALES = Marge / Recettes * 100.


ANALYSES OBLIGATOIRES
- Écart Réel/Prévision vs Budget ({current_year}).
- Évolution vs {current_year-1}.
- Analyse de la tendance {current_year-2}, {current_year-1} et {current_year}.
- Indicateur absent ou nul : écrire exactement « Non disponible ».

STYLE
- Français professionnel.
- Phrases courtes, factuelles.
- Toujours citer année et unité (€ ou %).
- TEXTE SIMPLE uniquement.

FORMAT DE SORTIE (STRICT)

**Résumé global** (4 lignes max)
- Dynamique générale (recettes vs charges).
- Écart majeur sur recettes ou charges ({current_year}).
- Lecture synthétique des marges.
- Appréciation opérationnelle.

**Risques principaux** (3 max)
- Valeur, année.

**Opportunités clés** (3 max)
- Valeur, année.

**Actions prioritaires** (3 max)
- Justification chiffrée.

**Conclusion de rentabilité** (En une phrase)
- Verdict clair : « Résidence rentable » ou « Résidence non rentable ».
- Justification chiffrée (marge, pourcentage des recettes, années).

CONTRAINTE FINALE
Respecte strictement la structure. Réponse concise.
"""

count_tokens(PROMPT_G)

1341

##### Recette

In [19]:
current_year = datetime.now().year

PROMPT_RECETTES = f"""
Tu es analyste financier senior spécialisé en pilotage du chiffre d'affaires de résidences étudiantes.

DONNÉES FOURNIES - RECETTES
{metric_text_ca}

RÈGLES STRICTES
- Aucune hypothèse ni extrapolation.
- Valeur prioritaire : Réel > Prévision > Budget.

RÈGLE RÉEL / PRÉVISION
- Janvier à août : Prévision = Réel.
- Septembre à décembre : Prévision = projection du Réel.

ANALYSES OBLIGATOIRES
- Écart Réel/Prévision vs Budget ({current_year}).
- Évolution {current_year} vs {current_year-1}.
- Valeur absente ou nulle : écrire « Non disponible ».

STYLE
- Français professionnel.
- Phrases courtes, factuelles.
- Années et unités obligatoires (€ / %).
- TEXTE SIMPLE uniquement.

FORMAT DE SORTIE (STRICT)

**Résumé recettes** (4 lignes max)
- Dynamique globale du chiffre d4affaires.
- Écart clé vs Budget ({current_year}).
- Variation vs {current_year-1}.
- Appréciation opérationnelle.

**Risques recettes** (3 max)
- Indicateur — valeur, année.

**Opportunités recettes** (3 max)
- Indicateur — valeur, année.

**Actions prioritaires** (3 max)
- Action — justification chiffrée.

CONTRAINTE FINALE
Respecte strictement la structure. Réponse concise.
"""
count_tokens(PROMPT_RECETTES)

1011

##### Charge

In [20]:
current_year = datetime.now().year

PROMPT_CHARGES = f"""
Tu es analyste financier senior spécialisé en pilotage des charges d'exploitation de résidences étudiantes.

DONNÉES FOURNIES - CHARGES
{metric_text_charge}

RÈGLES STRICTES
- Base-toi exclusivement sur les données fournies.
- Dernière valeur prioritaire : Réel > Prévision > Budget.

RÈGLE RÉEL / PRÉVISION
- Janvier à août : Prévision = Réel.
- Septembre à décembre : utiliser la Prévision.

ANALYSES OBLIGATOIRES
- Écart Réel/Prévision vs Budget ({current_year}).
- Évolution vs {current_year-1}.
- Valeur absente ou nulle : écrire exactement « Non disponible ».

STYLE
- Français professionnel.
- Phrases courtes.
- Toujours citer année et montant (€ ou %).
- TEXTE SIMPLE uniquement.

FORMAT DE SORTIE (STRICT)

**Résumé charges** (4 lignes max)
- Sens d'évolution global ({current_year}).
- Écart clé vs Budget.
- Poste le plus contributif.
- Appréciation opérationnelle.

**Postes sous surveillance** (3 max)
- Ecart (€), année.

**Opportunités d'optimisation** (3 max)
- Levier chiffré (€), année.

**Actions prioritaires** (3 max)
- Justification chiffrée.

CONTRAINTE FINALE
Respecte strictement la structure. Réponse concise.
"""

count_tokens(PROMPT_CHARGES)

835

### Test

##### Global

In [21]:
html_text_g, brt_g = await ollamaClient.ask_ollama(PROMPT_G)
display(HTML(html_text_g))

##### Recette

In [22]:
html_text_ca, brt_ca = await ollamaClient.ask_ollama(PROMPT_RECETTES)
display(HTML(html_text_ca))

##### Charge

In [33]:
html_text_charge, brt_charge = await ollamaClient.ask_ollama(PROMPT_CHARGES)
display(HTML(html_text_charge))

HTTPStatusError: Server error '502 Bad Gateway' for url 'http://si-5/api/chat'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502

In [24]:
df.iloc[0, 0]

'REA - Liste dynamique des résidences'

# BD

In [ ]:
brt_g = brt_g.replace("\u202f", "\\s")
brt_ca = brt_ca.replace("\u202f", "\\s")
brt_charge = brt_charge.replace("\u202f", "\\s")

res = await execute_sp(
    "dbo.sp_chatBotSAAnalyse_add",
    {
        "user_fk": config.USER_FK,
        "sa_fk": sa_fk,
        "analyseGlobal": brt_g,
        "analyseRecette": brt_ca,
        "analyseCharge": brt_charge
    },
    config.DATABASE_URL_IA
)

print("Sa_fk", "->", sa_fk)
print("Analyse Global", "->", len(brt_g))
print("Analyse Recette", "->", len(brt_ca))
print("Analyse Charge", "->", len(brt_charge))
print(res[0]["message"] if res else '')

Sa_fk -> 183
Analyse Global -> 1175
Analyse Recette -> 950
Analyse Charge -> 0
Enregistrement de l'analyse de la SA effectué avec succès


In [38]:
res = await execute_sp(
    "dbo.sp_chatBotSAAnalyse_get",
    {
        "user_fk": config.USER_FK,
        "sa_fk": sa_fk
    },
    config.DATABASE_URL_IA
)
if not res or not isinstance(res, list) or not res[0].get("analyseGlobal"):
    logger.warning(f"Aucune analyse générée pour le SA {sa_fk}.")
analyseGlobal: str = res[0]["analyseGlobal"]
analyseRecette: str = res[0]["analyseRecette"]
analyseCharge: str = res[0]["analyseCharge"]

In [40]:
analyseGlobal = analyseGlobal.replace("\\s", "\u202f")
analyseGlobal = format_response(analyseGlobal)
analyseGlobal = analyseGlobal.replace("\n", "")
display(HTML(analyseGlobal))

# Fin

---